In [47]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping
import string
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import joblib




Import file(s)

In [48]:
train_dataset = pd.read_csv("datasets/Labelled_1997_2017/HanDeSeT.csv")



In [49]:
train_dataset['Concat'] = train_dataset['utt1'].fillna('') + ' ' + train_dataset['utt2'].fillna('') + ' ' + train_dataset['utt3'].fillna('') + ' ' + train_dataset['utt4'].fillna('') + ' ' + train_dataset['utt5'].fillna('')



In [50]:
train_dataset = train_dataset.dropna(subset=['Concat'])
slimmed_train_dataset = train_dataset.drop (columns=['utt1', 'utt2', 'utt3', 'utt4', 'utt5', 'title', 'motion', 'id', 'party affiliation',])
slimmed_train_dataset.head()


,manual motion,govt/opp motion,motion party affiliation,manual speech,vote speech,name,rebellion %,Concat
0,1,1,Lab,0,0,Kevin McNamara,3.2,I trust that the House will not feel that ther...
1,1,1,Lab,0,0,John Swinney,0.4,The Secretary State talks of the lack of fuss ...
2,1,1,Lab,1,1,John McAllion,4.5,May I assure the right hon. Gentleman that he ...
3,1,1,Lab,0,1,Ian Davidson,3.1,"Is the hon. Gentleman aware that, ever since t..."
4,1,1,Lab,1,1,Maria Fyfe,2.3,The hon. Gentleman will notice that the Childr...


## Pre-processing

nlp training data split (train, test, validate)

In [51]:
test_dataset = slimmed_train_dataset[['manual speech', 'Concat']]


In [52]:

# Split the dataset into training and test sets (80% train, 20% test)
train_dataset, test_dataset = train_test_split(test_dataset, test_size=0.2, random_state=42)

# Further split the training set into training and validation sets (80% train, 20% validation)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Display the first few rows of each set
print("Training Set:")
print(train_dataset.head())
print("\nValidation Set:")
print(val_dataset.head())
print("\nTest Set:")
print(test_dataset.head())

Training Set:
      manual speech                                             Concat
1093              1  As mentioned in the debate, the Committees on ...
330               0  The hon. Lady has already conceded that she di...
1106              1  It is all very well looking at the cost of bui...
406               0  I thank the hon. Lady for giving way; she is b...
181               0  I wish to amplify the concern, raised by my ho...

Validation Set:
      manual speech                                             Concat
42                1  My constituency is far from London, but never ...
511               1  The Minister brushed aside the question from m...
546               1  Will the hon. Gentleman not acknowledge the tr...
470               1  As my hon. Friend knows, Northwick Park is jus...
1010              1  But does this not demonstrate why the Fixed-te...

Test Set:
      manual speech                                             Concat
721               1  I agree very m

Set up simple model so we can test on the whole dataset

In [53]:

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_dataset['Concat'])
X_val = vectorizer.transform(val_dataset['Concat'])
X_test = vectorizer.transform(test_dataset['Concat'])

# Get the target values
y_train = train_dataset['manual speech']
y_val = val_dataset['manual speech']
y_test = test_dataset['manual speech']

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model on the validation set
val_predictions = model.predict(X_val)
print("Validation Set Evaluation:")
print(classification_report(y_val, val_predictions))

# Evaluate the model on the test set
test_predictions = model.predict(X_test)
print("Test Set Evaluation:")
print(classification_report(y_test, test_predictions))

joblib.dump(model, 'logistic_regression_model.pkl')

'''
to load the model use 
model = joblib.load('logistic_regression_model.pkl')
'''



Validation Set Evaluation:
              precision    recall  f1-score   support

           0       0.63      0.36      0.45       104
           1       0.65      0.85      0.74       146

    accuracy                           0.64       250
   macro avg       0.64      0.60      0.59       250
weighted avg       0.64      0.64      0.62       250

Test Set Evaluation:
              precision    recall  f1-score   support

           0       0.73      0.38      0.50       114
           1       0.63      0.88      0.74       137

    accuracy                           0.65       251
   macro avg       0.68      0.63      0.62       251
weighted avg       0.67      0.65      0.63       251



"\nto load the model use \nmodel = joblib.load('logistic_regression_model.pkl')\n"

Train nlp model
If we are going to use the keras from the 

Assess model

At this point break out to another group to use trained model on nthe unlabelled data